# Final Methodology for CRMLS Building Complex Analysis
This notebook will layout the final methodology that is recommended to be used to identify the main building in a building complex. Given `crmls_units.csv` and `crmls_potential_unit_buildings.csv`, this method will result in a "high-confidence" match from the potential buildings dataset based on the total number of units at that address matches the `NumberofUnits` field. If it is unable to find this high-confidence match, it will then choose the row with the highest value in the fields that we care about. In order to confirm the `parcel_apn` of that building complex, a lookup table from the Lighthouse dataset needs to be made and used.

In [36]:
import pandas as pd
from ast import literal_eval
pd.set_option("display.max_columns", None)

## Load and Clean Data
The two datasets need to be loaded and the assessment data found in each row needs to be extracted.

In [37]:
try:
    units = pd.read_csv("data/clean_crmls_units.csv")
    p_buildings = pd.read_csv("data/clean_crmls_potential_unit_buildings.csv")
except FileNotFoundError:
    # crmls_units
    cr_units_raw = \
        pd.read_csv("data/crmls_units.csv",dtype=str,escapechar="\\")
    cr_units_raw['Assessments'] = cr_units_raw['Assessments'].apply(literal_eval)
    tempdf = pd.DataFrame(cr_units_raw['Assessments'].values.tolist(),index=cr_units_raw.index, columns=["test"])
    tempdf2 = tempdf['test'].apply(pd.Series)
    units = pd.concat([cr_units_raw, tempdf2],axis=1) #, ignore_index=True
    cr_pot_unit_blds_raw = \
        pd.read_csv("data/crmls_potential_unit_buildings.csv",escapechar="\\")
    cr_pot_unit_blds_raw['Assessments'] = cr_pot_unit_blds_raw['Assessments'].apply(literal_eval)
    tempdf = pd.DataFrame(cr_pot_unit_blds_raw['Assessments'].values.tolist(),index=cr_pot_unit_blds_raw.index, columns=["test"])
    tempdf2 = tempdf['test'].apply(pd.Series)
    p_buildings = pd.concat([cr_pot_unit_blds_raw, tempdf2],axis=1)
    del tempdf,tempdf2, cr_units_raw,cr_pot_unit_blds_raw

/var/folders/_3/v349phgn4ljd3jkf3ll9p2cm0000gp/T/ipykernel_75241/452960022.py:2: DtypeWarning: Columns (4,21) have mixed types. Specify dtype option on import or set low_memory=False.
  units = pd.read_csv("data/clean_crmls_units.csv")


## Getting High-Confidence Matches
A high confidence match is defined as the address matching as well as the total number of units in the building.

In [38]:
units_agg = units.groupby('ParcelFullAddress')['ogr_fid'].nunique().reset_index()
units_agg.columns = ['ParcelFullAddress', 'unit_count']

In [39]:
building_complex = p_buildings.merge(right=units_agg, right_on=['ParcelFullAddress', 'unit_count'], left_on=['ParcelFullAddress', 'NumberofUnits'])
bcom_stats = building_complex.groupby('ParcelFullAddress')['ogr_fid'].nunique()

In [40]:
bcom_stats[bcom_stats == 1]

ParcelFullAddress
1  BALDWIN AVE SAN MATEO CA 94401        1
1  GATE 6 RD SAUSALITO CA 94965          1
1  KELTON CT OAKLAND CA 94611            1
1  LONGVIEW DR WATSONVILLE CA 95076      1
1  SCOTTSDALE DR CARSON CA 90745         1
                                        ..
98  TYROL LN MAMMOTH LAKES CA 93546      1
9827 E LEMON AVE ARCADIA CA 91007        1
9873  ALDER AVE BLOOMINGTON CA 92316     1
99  MONTE CRESTA AVE OAKLAND CA 94611    1
999 W TAYLOR ST SAN JOSE CA 95126        1
Name: ogr_fid, Length: 1188, dtype: int64

In [41]:
high_con = building_complex[building_complex['ParcelFullAddress'].isin(bcom_stats[bcom_stats == 1].index)]
high_con['generated_building'] = [False] * len(high_con['ParcelFullAddress'])
high_con.columns

/var/folders/_3/v349phgn4ljd3jkf3ll9p2cm0000gp/T/ipykernel_75241/3992732294.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  high_con['generated_building'] = [False] * len(high_con['ParcelFullAddress'])


Index(['ogr_fid', 'ParcelFullAddress', 'stunitprfx', 'stunitnum', 'parcelapn',
       'taxapn', 'fips', 'parcelid', 'Upi', 'Assessments', 'AssesseeOwnerName',
       'AssesseeMail_CityName', 'AssesseeMail_StateCode', 'TotalAssessedValue',
       'AssessmentYear', 'TaxAmount', 'TaxYear', 'BuildingArea',
       'MainBuildingAreaIndicator', 'YearBuilt', 'NumberOfBuildings',
       'NumberOfStories', 'TotalNumberofRooms', 'NumberofUnits',
       'NumberofBedrooms', 'NumberofBaths', 'LatestValidPrice',
       'AssessedLandValue', 'AssessedImprovementValue', 'TaxRateCodeArea',
       'Owner1FirstName', 'Owner1LastName', 'Owner2Firstname',
       'Owner2MiddleName', 'Owner2LastName', 'ResoUPI', 'unit_count',
       'generated_building'],
      dtype='object')

In [42]:
needed_fields = ['ParcelFullAddress', 'parcelapn', 'taxapn', 'TotalAssessedValue', 'TaxAmount', 'BuildingArea', 'NumberOfBuildings', 'NumberofUnits', 'YearBuilt', 'NumberOfStories','TotalNumberofRooms', 'NumberofBedrooms', 'NumberofBaths', 'AssessedLandValue', 'AssessedImprovementValue', 'generated_building']

In [43]:
high_con[needed_fields]

,ParcelFullAddress,parcelapn,taxapn,TotalAssessedValue,TaxAmount,BuildingArea,NumberOfBuildings,NumberofUnits,YearBuilt,NumberOfStories,TotalNumberofRooms,NumberofBedrooms,NumberofBaths,AssessedLandValue,AssessedImprovementValue,generated_building
0,2741 GARBER ST BERKELEY CA 94705,54-1707-21,54-1707-21,0,NaN,7778,1,6,1924.0,3,28,9,600,0,0,False
1,2117 E AROMA DR WEST COVINA CA 91791,8493041072,8493-041-072,201087,260460.0,1231,0,1,1973.0,NaN,0,3,200,77107,123980,False
2,12 W RIANDA RD WATSONVILLE CA 95076,050-101-10,050-101-10,69551,103362.0,1266,0,1,1963.0,NaN,5,2,200,14665,54886,False
3,6672 BELL BLUFF AVE SAN DIEGO CA 92119,4562800532,456-280-05-32,180953,226004.0,806,0,1,1972.0,NaN,0,2,100,69595,111358,False
4,909 ALEJO VISTA PALM SPRINGS CA 92262,508038013,508-038-013,680068,902166.0,2258,0,1,2004.0,2,0,2,250,169875,510193,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4319,309 ORLANDO ST EL CAJON CA 92021,4891611500,489-161-15-00,1518662,3309226.0,0,0,37,NaN,NaN,0,0,0,1304585,214077,False
4320,1 BALDWIN AVE SAN MATEO CA 94401,109610010,109-610-010,4991946,140384.0,0,0,207,NaN,NaN,0,0,0,970861,4021085,False
4321,1165 HOLLISTER ST SAN DIEGO CA 92154,6340200300,634-020-03-00,1082217,1268484.0,0,0,55,NaN,NaN,0,0,0,811671,270546,False
4322,1201 PARKER DAM RD EARP CA 92242,066113104,0661-131-04-W001,1201406,1502593.0,2000,3,1,1963.0,1,0,0,0,131873,1069533,False


In [44]:
high_con[high_con['ParcelFullAddress'].str.contains('535')]

,ogr_fid,ParcelFullAddress,stunitprfx,stunitnum,parcelapn,taxapn,fips,parcelid,Upi,Assessments,AssesseeOwnerName,AssesseeMail_CityName,AssesseeMail_StateCode,TotalAssessedValue,AssessmentYear,TaxAmount,TaxYear,BuildingArea,MainBuildingAreaIndicator,YearBuilt,NumberOfBuildings,NumberOfStories,TotalNumberofRooms,NumberofUnits,NumberofBedrooms,NumberofBaths,LatestValidPrice,AssessedLandValue,AssessedImprovementValue,TaxRateCodeArea,Owner1FirstName,Owner1LastName,Owner2Firstname,Owner2MiddleName,Owner2LastName,ResoUPI,unit_count,generated_building
197,13537,3401 DALE RD MODESTO CA 95356,NaN,NaN,076031006000,076-031-006-000,6099,100660192_159381299,US-06099-N-076031006000-R-N,[{'AssesseeOwnerName': 'MACERICH VINTAGE FAIRE...,MACERICH VINTAGE FAIRE LTD PAR,SANTA MONICA,CA,10501411,2021,11411304.0,2021,9725,T,1980.0,0,1,0,1,0,0,0,597720,9903691,2-009,NaN,MACERICH VINTAGE FAIRE LTD PAR,NaN,NaN,NaN,US-06099-N-076031006000-R-N,1,False
201,73053,5351 BELGRAVE PL OAKLAND CA 94618,NaN,NaN,48A-7034-50,48A-7034-50,6001,100660192_173200987,US-06001-N-48A703450-R-N,"[{'AssesseeOwnerName': 'HOMEOWNERS ASSOC', 'As...",HOMEOWNERS ASSOC,OAKLAND,CA,0,2021,NaN,2021,13458,L,1979.0,1,3,39,9,0,1500,0,0,0,17-001,NaN,HOMEOWNERS ASSOC,NaN,NaN,NaN,US-06001-N-48A703450-R-N,9,False
310,67946,43141 BUSINESS CENTER PKWY LANCASTER CA 93535,NaN,NaN,3126027138,3126-027-138,6037,100660192_193663029,US-06037-N-3126027138-R-N,[{'AssesseeOwnerName': 'BCP COMMERCIAL PROPERT...,BCP COMMERCIAL PROPERTIES LLC,LANCASTER,CA,5309604,2021,8488900.0,2021,63029,T,2002.0,0,1,0,1,0,0,0,394050,4915554,5-726,NaN,BCP COMMERCIAL PROPERTIES LLC,NaN,NaN,NaN,US-06037-N-3126027138-R-N,1,False
2233,40,535 PIERCE ST ALBANY CA 94706,NaN,NaN,66-2761-129,66-2761-129,6001,100660192_173145050,US-06001-N-662761129-R-N,[{'AssesseeOwnerName': 'COMMON ARE OF TR5873 (...,COMMON ARE OF TR5873 (66-2763) 1THRU235 & 131T...,BURLINGAME,CA,0,2021,NaN,2021,317904,L,1988.0,1,4,0,235,0,0,221000,0,0,22-000,NaN,NaN,NaN,NaN,NaN,US-06001-N-662761129-R-N,235,False
2453,60508,1119 12TH ST MODESTO CA 95354,NaN,NaN,105028003000,105-028-003-000,6099,100660192_159426947,US-06099-N-105028003000-R-N,"[{'AssesseeOwnerName': 'GOSS MICHAEL A', 'Asse...",GOSS MICHAEL A,MODESTO,CA,481139,2021,603276.0,2021,3780,T,1963.0,0,1,0,1,0,0,0,159000,322139,2-173,MICHAEL A,GOSS,NaN,NaN,NaN,US-06099-N-105028003000-R-N,1,False
2471,20201,1620 N CARPENTER RD MODESTO CA 95351,NaN,NaN,029008018000,029-008-018-000,6099,100660192_159387698,US-06099-N-029008018000-R-N,"[{'AssesseeOwnerName': 'CAMPUS COMMONS LLC', '...",CAMPUS COMMONS LLC,MODESTO,CA,751650,2021,821802.0,2021,9687,T,1983.0,0,2,0,4,0,0,0,155190,596460,2-173,NaN,CAMPUS COMMONS LLC,NaN,NaN,NaN,US-06099-N-029008018000-R-N,4,False
2919,331262,5357 HOWARD ST ONTARIO CA 91762,NaN,NaN,101150111,1011-501-11-0000,6071,100660192_183595738,US-06071-N-1011501110000-R-N,[{'AssesseeOwnerName': 'GAZZOLO FAMILY TRUST (...,GAZZOLO FAMILY TRUST (6-26-03),RIVERSIDE,CA,76934,2021,89535.0,2021,1728,T,1964.0,0,NaN,0,2,0,0,162000,30775,46159,98-000,NaN,GAZZOLO FAMILY TRUST,NaN,NaN,NaN,US-06071-N-1011501110000-R-N,2,False
2935,58718,1507 9TH ST MODESTO CA 95354,NaN,NaN,104001020000,104-001-020-000,6099,100660192_159425234,US-06099-N-104001020000-R-N,"[{'AssesseeOwnerName': 'BAKER LEE R', 'Assesse...",BAKER LEE R,CERES,CA,504000,2021,551038.0,2021,7200,T,1962.0,0,1,0,1,0,0,0,151000,353000,2-173,RODRICK L,BAKER,SHARON L,NaN,BAKER,US-06099-N-104001020000-R-N,1,False
2960,305392,535 N HIGHWAY 101 SOLANA BEACH CA 92075,NaN,NaN,2630430300,263-043-03-00,6073,100660192_186977666,US-06073-N-2630430300-R-N,"[{'AssesseeOwnerName': 'R S R PROPERTIES LLC',...",R S R PROPERTIES LLC,SOLANA BEACH,CA,780986,2021,1064138.0,2021,5000,T,1980.0,0,NaN,0,1,0,0,0,597023,183963,18-117,NaN,RSR PROPERTIES LLC,NaN,NaN,NaN,US-06073-N-2630430300-R-N,1,False
3167,11060,3801 PELANDALE AVE MODESTO CA 95356,NaN,NaN,078014050000,078-014-050-000,6099,100660192_159379031,US-06099-N-078014050000-R-N,[{'AssesseeOwnerName': 'COSTCO WHOLESAL

In [45]:
high_con_units = units[units['ParcelFullAddress'].isin(high_con['ParcelFullAddress'])]
high_con_units.shape

(15739, 36)

## Creating New Buildings
For the buildings we were unable to identify, we are creating building complexes for them from the units data.


In [46]:
units_low_con = units[~units['ParcelFullAddress'].isin(high_con['ParcelFullAddress'])]

In [47]:
units_low_con.shape

(1058251, 36)

In [48]:
new_complex = units_low_con.groupby('ParcelFullAddress')['ogr_fid'].nunique()

In [49]:
new_complex[new_complex > 1] # should we only create buildings with multiple units?

ParcelFullAddress
1  1ST ST LAKEPORT CA 95453                      3
1  3RD PL LONG BEACH CA 90802                   42
1  62ND PL LONG BEACH CA 90803                  12
1  ANDREW DR BELVEDERE TIBURON CA 94920          5
1  ANDREW DR TIBURON CA 94920                    3
                                                ..
SOLSTICE AVE CHULA VISTA CA 91915               12
SUGAR BUSH CT CHULA VISTA CA 91915               2
THE EMBARCADERO PIER  SAN FRANCISCO CA 94111     2
VIA CANDIDIZ  SAN DIEGO CA 92130                 2
W HARBOR DR SAN DIEGO CA 92101                   2
Name: ogr_fid, Length: 74085, dtype: int64

In [50]:
orphaned = new_complex[new_complex == 1] # should we only create buildings with multiple units?
print(orphaned)
orphaned_units = units[units['ParcelFullAddress'].isin(orphaned.index)]
orphaned_units.groupby('stunitprfx')['ogr_fid'].nunique()

ParcelFullAddress
094 # 94 OFC HWY CAMPO CA 91906          1
1    KETTLEMAN CITY CA 93239             1
1    LAKESIDE CA 92040                   1
1  ABBEY LN ALISO VIEJO CA 92656         1
1  ACORN  IRVINE CA 92604                1
                                        ..
W11-B  HARBOR DR SAN DIEGO CA 92101      1
W14-F1  WESTERN ST SAN DIEGO CA 92101    1
W4-C  WESTERN ST SAN DIEGO CA 92027      1
W7-A  WESTERN ST SAN DIEGO CA 92101      1
WEST OF HWY   93526                      1
Name: ogr_fid, Length: 95386, dtype: int64


stunitprfx
#       81236
APT      7879
BLDG       30
FL         10
FRNT       16
HNGR        3
KEY         1
LOT        45
LOWR        1
OFC        37
PH          2
PIER       13
RM          1
SIDE       51
SPC       181
STE      2525
STOP        3
TRLR        7
UNIT      808
UPPR        9
Name: ogr_fid, dtype: int64

In [51]:
buildings = pd.DataFrame({'ParcelFullAddress': new_complex.index, 'NumberofUnits': new_complex.values, 'generated_building': [True] * len(new_complex)})
buildings

,ParcelFullAddress,NumberofUnits,generated_building
0,094 # 94 OFC HWY CAMPO CA 91906,1,True
1,1 KETTLEMAN CITY CA 93239,1,True
2,1 LAKESIDE CA 92040,1,True
3,1 1ST ST LAKEPORT CA 95453,3,True
4,1 3RD PL LONG BEACH CA 90802,42,True
...,...,...,...
169466,W11-B HARBOR DR SAN DIEGO CA 92101,1,True
169467,W14-F1 WESTERN ST SAN DIEGO CA 92101,1,True
169468,W4-C WESTERN ST SAN DIEGO CA 92027,1,True
169469,W7-A WESTERN ST SAN DIEGO CA 92101,1,True


In [52]:
high_con = high_con[needed_fields]
pd.concat([high_con, buildings]).to_csv('data/buildings.csv', index=False)

## Parcelapn investigation

In [53]:
units[units['ParcelFullAddress'] == '6400  SURFSIDE LN CARLSBAD CA 92011']

,ogr_fid,ParcelFullAddress,stunitprfx,stunitnum,parcelapn,taxapn,fips,parcelid,Upi,Assessments,AssesseeOwnerName,AssesseeMail_CityName,AssesseeMail_StateCode,TotalAssessedValue,AssessmentYear,TaxAmount,TaxYear,BuildingArea,MainBuildingAreaIndicator,YearBuilt,NumberOfBuildings,NumberOfStories,TotalNumberofRooms,NumberofUnits,NumberofBedrooms,NumberofBaths,LatestValidPrice,AssessedLandValue,AssessedImprovementValue,TaxRateCodeArea,Owner1FirstName,Owner1LastName,Owner2Firstname,Owner2MiddleName,Owner2LastName,ResoUPI
706371,254651,6400 SURFSIDE LN CARLSBAD CA 92011,#,308DO,2140119036,214-011-90-36,6073,100660192_186927719,US-06073-N-2140119036-R-N,[{'AssesseeOwnerName': 'GRAND PACIFIC CARLSBAD...,GRAND PACIFIC CARLSBAD LP,CARLSBAD,CA,0,2021,NaN,2021,577,L,1997.0,0,NaN,0,1,0,0,0,0,0,9-018,NaN,GRAND PACIFIC CARLSBAD LP,NaN,NaN,NaN,US-06073-N-2140119036-R-N


In [54]:
units[units['stunitprfx'] == 'PIER']

,ogr_fid,ParcelFullAddress,stunitprfx,stunitnum,parcelapn,taxapn,fips,parcelid,Upi,Assessments,AssesseeOwnerName,AssesseeMail_CityName,AssesseeMail_StateCode,TotalAssessedValue,AssessmentYear,TaxAmount,TaxYear,BuildingArea,MainBuildingAreaIndicator,YearBuilt,NumberOfBuildings,NumberOfStories,TotalNumberofRooms,NumberofUnits,NumberofBedrooms,NumberofBaths,LatestValidPrice,AssessedLandValue,AssessedImprovementValue,TaxRateCodeArea,Owner1FirstName,Owner1LastName,Owner2Firstname,Owner2MiddleName,Owner2LastName,ResoUPI
432366,148569,11300 PIERO STANTON CA 90680,PIER,O,937-67-614,937-676-14,6059,519885940_3505,US-06059-N-93767614-R-N,"[{'AssesseeOwnerName': 'PHAN UT THI', 'Assesse...",PHAN UT THI,WESTMINSTER,CA,343243,2021,425422.0,2021,1442,L,2007.0,0,NaN,0,0,2,200,290000,124176,219067,17-055,UT THI,PHAN,NaN,NaN,NaN,US-06059-N-93767614-R-N
432367,148574,11304 PIERO STANTON CA 90680,PIER,O,937-67-615,937-676-15,6059,519885940_3506,US-06059-N-93767615-R-N,[{'AssesseeOwnerName': 'RODRIGUEZ ANNABELLE M ...,RODRIGUEZ ANNABELLE M TR,STANTON,CA,470000,2021,557916.0,2021,1389,L,2007.0,0,NaN,0,0,2,200,640500,259544,210456,17-055,KRISTIN AKEMI,NAKANO,NaN,NaN,NaN,US-06059-N-93767615-R-N
432368,148607,11305 PIERO STANTON CA 90680,PIER,O,937-67-600,937-676-00,6059,519885940_3515,US-06059-N-93767600-R-N,[{'AssesseeOwnerName': 'PROBART THOMAS ARTHUR ...,PROBART THOMAS ARTHUR & ELYSSA JANELLE,STANTON,CA,442925,2021,527960.0,2021,1389,L,2007.0,0,NaN,0,0,2,200,405000,230386,212539,17-055,THOMAS ARTHUR,PROBART,ELYSSA JANELLE,NaN,PROBART,US-06059-N-93767600-R-N
432369,148612,11301 PIERO STANTON CA 90680,PIER,O,937-67-601,937-676-01,6059,519885940_3516,US-06059-N-93767601-R-N,[{'AssesseeOwnerName': 'DUONG JULIA N PHAN & B...,DUONG JULIA N PHAN & BRIAN C,STANTON,CA,475000,2021,571192.0,2021,1442,L,2007.0,0,NaN,0,0,2,200,477500,261187,213813,17-055,JULIA N PHAN,DUONG,BRIAN C,NaN,DUONG,US-06059-N-93767601-R-N
621367,2557,706 AVENIDA DEL PIERO SAN JUAN BAUTISTA CA 9...,PIER,O,122500010,012-250-001-000,6069,100660192_205071848,US-06069-N-012250001000-R-N,"[{'AssesseeOwnerName': 'OTERO PAUL & TAMMY', '...",OTERO PAUL & TAMMY,SAN JUAN BAUTISTA,CA,764436,2021,884002.0,2021,3146,L,2004.0,0,1,0,0,3,300,675000,368061,396375,79-032,PAUL,OTERO,TAMMY,NaN,OTERO,US-06069-N-012250001000-R-N
621368,2558,729 AVENIDA DEL PIERO SAN JUAN BAUTISTA CA 9...,PIER,O,122500040,012-250-004-000,6069,100660192_205071849,US-06069-N-012250004000-R-N,"[{'AssesseeOwnerName': 'LEEMASTER JACK-KAREN',...",LEEMASTER JACK-KAREN,SAN JUAN BAUTISTA,CA,950000,2021,1091366.0,2021,3415,L,2005.0,0,1,0,0,3,200,982500,350000,600000,79-032,JACK,LEEMASTER,KAREN,NaN,LEESMATER,US-06069-N-012250004000-R-N
621369,2559,759 AVENIDA DEL PIERO SAN JUAN BAUTISTA CA 9...,PIER,O,122500060,012-250-006-000,6069,100660192_205071850,US-06069-N-012250006000-R-N,[{'AssesseeOwnerName': 'GARIBAY PEDRO-MARIA D'...,GARIBAY PEDRO-MARIA D,SAN JUAN BAUTISTA,CA,950000,2021,1089766.0,2021,3415,L,2005.0,0,1,0,0,3,200,1020500,350000,600000,79-032,PEDRO,GARIBAY,MARIA D,NaN,GARIBAY,US-06069-N-012250006000-R-N
621370,2560,716 AVENIDA DEL PIERO SAN JUAN BAUTISTA CA 9...,PIER,O,122500020,012-250-002-000,6069,100660192_205071851,US-06069-N-012250002000-R-N,[{'AssesseeOwnerName': 'REA RICHARD WILLIAM 20...,REA RICHARD WILLIAM 2008 REV TR,SAN JOSE,CA,922091,2021,1065064.0,2021,3146,L,2005.0,0,1,0,0,3,300,860000,375269,546822,79-032,NaN,REA RICHARD WILLIAM 2008 REV TR,NaN,NaN,NaN,US-06069-N-012250002000-R-N
621371,2561,726 AVENIDA DEL PIERO SAN JUAN BAUTISTA CA 9...,PIER,O,122500030,012-250-003-000,6069,100660192_205071852,US-06069-N-012250003000-R-N,[{'AssesseeOwnerName': 'THOME PHIL-CLAIRE REV ...,THOME PHIL-CLAIRE REV TR,SAN JUAN BAUTISTA,CA,820231,2021,940230.0,2021,3146,L,2005.0,0,1,0,0,3,300,750000,382774,437457,79-032,CLAIRE C,THOME,PHIL R,NaN,THOME,US-06069-N-012250003000-R-N
621372,2562,756 AVENIDA DEL PIERO SAN JUAN BAUTISTA CA 9...,PIER,O,122500070,012-250-007-000,6069,100660192_205071853,US-06069-N-012250007000-R-N,[{'Assessee

In [55]:
p_buildings[p_buildings['ParcelFullAddress'].str.contains('PIER ')]

,ogr_fid,ParcelFullAddress,stunitprfx,stunitnum,parcelapn,taxapn,fips,parcelid,Upi,Assessments,AssesseeOwnerName,AssesseeMail_CityName,AssesseeMail_StateCode,TotalAssessedValue,AssessmentYear,TaxAmount,TaxYear,BuildingArea,MainBuildingAreaIndicator,YearBuilt,NumberOfBuildings,NumberOfStories,TotalNumberofRooms,NumberofUnits,NumberofBedrooms,NumberofBaths,LatestValidPrice,AssessedLandValue,AssessedImprovementValue,TaxRateCodeArea,Owner1FirstName,Owner1LastName,Owner2Firstname,Owner2MiddleName,Owner2LastName,ResoUPI
11514,113877,444 PIER AVE OCEANO CA 93445,NaN,NaN,061-033-057,061-033-057,6079,100660192_15370677,US-06079-N-061033057-R-N,[{'AssesseeOwnerName': 'PACIFIC PLAZA RESORT O...,PACIFIC PLAZA RESORT OWNERS ASSOCIATION,ARROYO GRANDE,CA,0,2021,NaN,2021,0,NaN,NaN,0,NaN,0,0,0,0,612000,0,0,52-054,NaN,PACIFIC PLAZA RESORT OWNERS ASSOCIATION,NaN,NaN,NaN,US-06079-N-061033057-R-N
16992,97114,1 JOHNSON PIER HALF MOON BAY CA 94019,NaN,NaN,047390020,3047FH,6081,100660192_163247943,US-06081-N-3047FH-R-N,[{'AssesseeOwnerName': 'PLYMPTON CHRISTOPHER L...,PLYMPTON CHRISTOPHER LANCE,EL GRANADA,CA,4505,2021,NaN,2021,0,NaN,NaN,0,NaN,0,0,0,0,0,4505,0,87-022,CHRISTOPHER,PLYMPTON,NaN,NaN,NaN,US-06081-N-3047FH-R-N
16993,97115,1 JOHNSON PIER HALF MOON BAY CA 94019,NaN,NaN,047390020,047-390-020,6081,100660192_163247943,US-06081-N-047390020-R-N,[{'AssesseeOwnerName': 'SAN MATEO COUNTY HARBO...,SAN MATEO COUNTY HARBOR DIST,EL GRANADA,CA,0,2021,1868.0,2021,0,NaN,NaN,0,NaN,0,0,0,0,0,0,0,87-022,NaN,SAN MATEO COUNTY HARBOR DIST,NaN,NaN,PILLAR POINT HARBOR,US-06081-N-047390020-R-N
16994,97116,1 JOHNSON PIER HALF MOON BAY CA 94019,NaN,NaN,047390020,1107987,6081,100660192_163247943,US-06081-N-1107987-R-N,"[{'AssesseeOwnerName': 'HANSEN ROBERT', 'Asses...",HANSEN ROBERT,HALF MOON BAY,CA,2856,2021,NaN,2021,0,NaN,NaN,0,NaN,0,0,0,0,0,2856,0,87-022,ROBERT,HANSEN,NaN,NaN,NaN,US-06081-N-1107987-R-N
16995,97117,1 JOHNSON PIER HALF MOON BAY CA 94019,NaN,NaN,047390020,1145163,6081,100660192_163247943,US-06081-N-1145163-R-N,"[{'AssesseeOwnerName': 'FISHER FREDERICK W', '...",FISHER FREDERICK W,PACIFICA,CA,2562,2021,NaN,2021,0,NaN,NaN,0,NaN,0,0,0,0,0,2562,0,87-022,FREDERICK,FISHER,NaN,NaN,NaN,US-06081-N-1145163-R-N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17061,97200,1 JOHNSON PIER HALF MOON BAY CA 94019,NaN,NaN,047390020,1026PU,6081,100660192_163247943,US-06081-N-1026PU-R-N,"[{'AssesseeOwnerName': 'COTTA RON', 'AssesseeM...",COTTA RON,HALF MOON BAY,CA,6383,2021,NaN,2021,0,NaN,NaN,0,NaN,0,0,0,0,0,6383,0,17-000,RON,COTTA,NaN,NaN,NaN,US-06081-N-1026PU-R-N
17062,97201,1 JOHNSON PIER HALF MOON BAY CA 94019,NaN,NaN,047390020,1545GA,6081,100660192_163247943,US-06081-N-1545GA-R-N,"[{'AssesseeOwnerName': 'CLAY BEATTY', 'Assesse...",CLAY BEATTY,EL GRANADA,CA,4842,2021,NaN,2021,0,NaN,NaN,0,NaN,0,0,0,0,0,4842,0,17-000,CLAY,BEATTY,NaN,NaN,NaN,US-06081-N-1545GA-R-N
17063,97202,1 JOHNSON PIER HALF MOON BAY CA 94019,NaN,NaN,047390020,5336DY,6081,100660192_163247943,US-06081-N-5336DY-R-N,"[{'AssesseeOwnerName': 'WESLEY JESSICA', 'Asse...",WESLEY JESSICA,EL GRANADA,CA,6383,2021,NaN,2021,0,NaN,NaN,0,NaN,0,0,0,0,0,6383,0,87-022,JESSICA,WESLEY,NaN,NaN,NaN,US-06081-N-5336DY-R-N
17064,97203,1 JOHNSON PIER HALF MOON BAY CA 94019,NaN,NaN,047390020,7582EP,6081,100660192_163247943,US-06081-N-7582EP-R-N,[{'AssesseeOwnerName': 'OBRIEN JAMES STEVENSON...,OBRIEN JAMES STEVENSON,KENSINGTON,CA,2848,2021,NaN,2021,0,NaN,NaN,0,NaN,0,0,0,0,0,2848,0,87-022,JAMES,OBRIEN,NaN,NaN,NaN,US-06081-N-7582EP-R-N


In [56]:
p_buildings.groupby('parcelapn')['ParcelFullAddress'].nunique()[p_buildings.groupby('parcelapn')['ParcelFullAddress'].nunique() > 1]

parcelapn
013046130          2
017017104          2
020212115          2
025121121          2
030-200-092-000    2
                  ..
815032007          4
815240003          2
84105013           2
931-33-122         2
934-04-142         3
Name: ParcelFullAddress, Length: 62, dtype: int64

In [57]:
p_buildings.groupby('parcelapn')['ParcelFullAddress'].nunique()[p_buildings.groupby('parcelapn')['ParcelFullAddress'].nunique() > 1]

parcelapn
013046130          2
017017104          2
020212115          2
025121121          2
030-200-092-000    2
                  ..
815032007          4
815240003          2
84105013           2
931-33-122         2
934-04-142         3
Name: ParcelFullAddress, Length: 62, dtype: int64

In [86]:
p_buildings[p_buildings['parcelapn'] == '815032007']

,ogr_fid,ParcelFullAddress,stunitprfx,stunitnum,parcelapn,taxapn,fips,parcelid,Upi,Assessments,AssesseeOwnerName,AssesseeMail_CityName,AssesseeMail_StateCode,TotalAssessedValue,AssessmentYear,TaxAmount,TaxYear,BuildingArea,MainBuildingAreaIndicator,YearBuilt,NumberOfBuildings,NumberOfStories,TotalNumberofRooms,NumberofUnits,NumberofBedrooms,NumberofBaths,LatestValidPrice,AssessedLandValue,AssessedImprovementValue,TaxRateCodeArea,Owner1FirstName,Owner1LastName,Owner2Firstname,Owner2MiddleName,Owner2LastName,ResoUPI
23340,983363,30480 HIGHWAY 95 BLYTHE CA 92225,NaN,NaN,815032007,009-606-655,6065,100660192_185740415,US-06065-N-009606655-R-N,"[{'AssesseeOwnerName': 'LAIRD BRIAN', 'Assesse...",LAIRD BRIAN,OAK GLEN,CA,235542,2021,258958.0,2021,0,NaN,NaN,0,NaN,0,0,0,0,0,26673,208869,85-000,BRIAN,LAIRD,NaN,NaN,NaN,US-06065-N-009606655-R-N
32695,983410,1700 HWY 95 BLYTHE CA 92225,NaN,NaN,815032007,009-619-371,6065,100660192_185740415,US-06065-N-009619371-R-N,"[{'AssesseeOwnerName': 'LYON R KEITH', 'Assess...",LYON R KEITH,FONTANA,CA,153965,2021,169270.0,2021,1586,L,2013.0,0,1,0,0,3,200,0,36189,117776,85-000,NaN,NaN,NaN,NaN,NaN,US-06065-N-009619371-R-N
41594,983354,30500 US HIGHWAY 95 BLYTHE CA 92225,NaN,NaN,815032007,009-601-675,6065,100660192_185740415,US-06065-N-009601675-R-N,"[{'AssesseeOwnerName': 'THWEATT ROBERT', 'Asse...",THWEATT ROBERT,BLYTHE,CA,163518,2021,179772.0,2021,0,NaN,NaN,0,NaN,0,46,0,0,0,36846,126672,85-000,ROBERT,THWEATT,NaN,NaN,NaN,US-06065-N-009601675-R-N
41595,983393,30500 US HIGHWAY 95 BLYTHE CA 92225,NaN,NaN,815032007,009-703-898,6065,100660192_185740415,US-06065-N-009703898-R-N,"[{'AssesseeOwnerName': 'JOHNSON TERRY', 'Asses...",JOHNSON TERRY,BLYTHE,CA,15531,2021,17074.0,2021,0,NaN,NaN,0,NaN,0,0,0,0,0,0,15531,85-000,TERRY,JOHNSON,NaN,NaN,NaN,US-06065-N-009703898-R-N
41596,983402,30500 US HIGHWAY 95 BLYTHE CA 92225,NaN,NaN,815032007,009-714-983,6065,100660192_185740415,US-06065-N-009714983-R-N,"[{'AssesseeOwnerName': 'FRITZ MARC J', 'Assess...",FRITZ MARC J,GARDEN GROVE,CA,54906,2021,60364.0,2021,1736,L,1993.0,0,1,0,0,3,200,0,0,54906,85-000,MARC,FRITZ,FRITZ,M,BAILEY,US-06065-N-009714983-R-N
41597,983406,30500 US HIGHWAY 95 BLYTHE CA 92225,NaN,NaN,815032007,815-032-007,6065,100660192_185740415,US-06065-N-815032007-R-N,"[{'AssesseeOwnerName': 'USA BIA', 'AssesseeMai...",USA BIA,NaN,NaN,0,2021,NaN,2021,0,NaN,NaN,0,NaN,0,0,0,0,0,0,0,85-000,BIA,USA,NaN,NaN,NaN,US-06065-N-815032007-R-N
44070,983364,31800 HIGHWAY 95 BLYTHE CA 92225,NaN,NaN,815032007,009-606-664,6065,100660192_185740415,US-06065-N-009606664-R-N,"[{'AssesseeOwnerName': 'LEWIS DUANE', 'Assesse...",LEWIS DUANE,SIMI VALLEY,CA,29006,2021,31890.0,2021,0,NaN,NaN,0,NaN,0,0,0,0,0,26672,2334,85-000,DUANE,LEWIS,NaN,NaN,NaN,US-06065-N-009606664-R-N
44071,983365,31800 HIGHWAY 95 BLYTHE CA 92225,NaN,NaN,815032007,009-606-665,6065,100660192_185740415,US-06065-N-009606665-R-N,"[{'AssesseeOwnerName': 'JORDAN JAMES', 'Assess...",JORDAN JAMES,SAN DIEGO,CA,14871,2021,16348.0,2021,0,NaN,NaN,0,NaN,0,0,0,0,0,12537,2334,85-000,JAMES,JORDAN,NaN,NaN,NaN,US-06065-N-009606665-R-N


In [59]:
p_buildings[p_buildings['ParcelFullAddress'].str.contains('SCHUBERT CT')]

,ogr_fid,ParcelFullAddress,stunitprfx,stunitnum,parcelapn,taxapn,fips,parcelid,Upi,Assessments,AssesseeOwnerName,AssesseeMail_CityName,AssesseeMail_StateCode,TotalAssessedValue,AssessmentYear,TaxAmount,TaxYear,BuildingArea,MainBuildingAreaIndicator,YearBuilt,NumberOfBuildings,NumberOfStories,TotalNumberofRooms,NumberofUnits,NumberofBedrooms,NumberofBaths,LatestValidPrice,AssessedLandValue,AssessedImprovementValue,TaxRateCodeArea,Owner1FirstName,Owner1LastName,Owner2Firstname,Owner2MiddleName,Owner2LastName,ResoUPI
2726,620262,19 SCHUBERT CT IRVINE CA 92617,NaN,NaN,455-161-20,988-882-02,6059,100660192_215802407,US-06059-N-98888202-R-N,[{'AssesseeOwnerName': 'KUZNETSOV YURII TR GEO...,KUZNETSOV YURII TR GEORGIEVICH FAMILY TR,IRVINE,CA,199358,2021,209016.0,2021,1097,L,1986.0,0,NaN,0,0,2,100,144000,76506,122852,26-006,YURII,KUZNETSOV,GOARK,NaN,MELKONYAN,US-06059-N-98888202-R-N
28356,619776,133 SCHUBERT CT IRVINE CA 92617,NaN,NaN,934-04-142,988-970-01,6059,510371586_185719,US-06059-N-98897001-R-N,"[{'AssesseeOwnerName': 'BALU MIHAELA', 'Assess...",BALU MIHAELA,IRVINE,CA,201776,2021,211500.0,2021,812,L,1987.0,0,NaN,0,0,1,100,183000,118367,83409,26-008,MIHAELA,BALU,NaN,NaN,NaN,US-06059-N-98897001-R-N
35355,619775,135 SCHUBERT CT IRVINE CA 92617,NaN,NaN,934-04-142,988-895-01,6059,510371586_185719,US-06059-N-98889501-R-N,"[{'AssesseeOwnerName': 'CURRIE ELLIOTT TR', 'A...",CURRIE ELLIOTT TR,BERKELEY,CA,177871,2021,194144.0,2021,972,L,1987.0,0,NaN,0,0,1,100,133500,63695,114176,26-008,ELLIOTT,CURRIE,RACHAEL A,NaN,PELTZ,US-06059-N-98889501-R-N
40403,619756,69 SCHUBERT CT IRVINE CA 92617,NaN,NaN,934-04-142,988-041-41,6059,510371586_185719,US-06059-N-98804141-R-N,[{'AssesseeOwnerName': 'REGENTS OF THE UNIVERS...,REGENTS OF THE UNIVERSITY OF CALIFORNIA,IRVINE,CA,197803,2021,210198.0,2021,812,L,1987.0,0,NaN,0,0,1,100,168500,106591,91212,26-008,OF,REGENTS OF THE UNIVERSITY,NaN,NaN,NaN,US-06059-N-98804141-R-N


In [87]:
units[units['ParcelFullAddress'].str.contains('HIGHWAY 95')]

,ogr_fid,ParcelFullAddress,stunitprfx,stunitnum,parcelapn,taxapn,fips,parcelid,Upi,Assessments,AssesseeOwnerName,AssesseeMail_CityName,AssesseeMail_StateCode,TotalAssessedValue,AssessmentYear,TaxAmount,TaxYear,BuildingArea,MainBuildingAreaIndicator,YearBuilt,NumberOfBuildings,NumberOfStories,TotalNumberofRooms,NumberofUnits,NumberofBedrooms,NumberofBaths,LatestValidPrice,AssessedLandValue,AssessedImprovementValue,TaxRateCodeArea,Owner1FirstName,Owner1LastName,Owner2Firstname,Owner2MiddleName,Owner2LastName,ResoUPI,key
609971,982350,42500 HIGHWAY 95 BLYTHE CA 92225,#,307,806242004,009-703-727,6065,100660192_185740119,US-06065-N-009703727-R-N,"[{'AssesseeOwnerName': 'DZIDA GEORGE A', 'Asse...",DZIDA GEORGE A,REDLANDS,CA,24602,2021,27048.0,2021,1248,L,1981.0,0,1,0,0,3,200,0,0,24602,85-000,GEORGE,DZIDA,CHRIS,R,DZIDA,US-06065-N-009703727-R-N,806242
609972,982351,47500 HIGHWAY 95 BLYTHE CA 92225,#,320,806242004,009-704-676,6065,100660192_185740119,US-06065-N-009704676-R-N,"[{'AssesseeOwnerName': 'LEWIS BUDDY L', 'Asses...",LEWIS BUDDY L,ALTA LOMA,CA,35262,2021,38766.0,2021,896,L,1982.0,0,1,0,0,2,100,0,0,35262,85-000,BUDDY,LEWIS,NaN,NaN,NaN,US-06065-N-009704676-R-N,806242
609973,982352,47500 HIGHWAY 95 BLYTHE CA 92225,#,342,806242004,009-706-287,6065,100660192_185740119,US-06065-N-009706287-R-N,"[{'AssesseeOwnerName': 'DEBRUYN RONALD M', 'As...",DEBRUYN RONALD M,PARKER,AZ,9609,2021,10564.0,2021,771,L,1967.0,0,1,0,0,2,100,0,0,9609,85-000,RONALD,DEBRUYN,SANDRA,NaN,DEBRUYN,US-06065-N-009706287-R-N,806242
609974,982353,47500 HIGHWAY 95 BLYTHE CA 92225,#,317,806242004,009-706-288,6065,100660192_185740119,US-06065-N-009706288-R-N,"[{'AssesseeOwnerName': 'MYKLAK JOHN', 'Assesse...",MYKLAK JOHN,GARDEN GROVE,CA,22850,2021,25122.0,2021,924,L,1982.0,0,1,0,0,2,100,0,0,22850,85-000,JOHN,MYKLAK,NaN,NaN,NaN,US-06065-N-009706288-R-N,806242
609975,982354,42500 HIGHWAY 95 BLYTHE CA 92225,#,333,806242004,009-707-311,6065,100660192_185740119,US-06065-N-009707311-R-N,[{'AssesseeOwnerName': 'MURRAY RANDALL M TRUST...,MURRAY RANDALL M TRUST,YUCAIPA,CA,25549,2021,28088.0,2021,792,L,1978.0,0,1,0,0,2,100,0,0,25549,85-000,NaN,MURRAY RANDALL M TRUST,NaN,NaN,NaN,US-06065-N-009707311-R-N,806242
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
610607,983605,1300 HIGHWAY 95 BLYTHE CA 92225,#,65,815240003,009-715-227,6065,100660192_185740491,US-06065-N-009715227-R-N,"[{'AssesseeOwnerName': 'COURSON JAMES E', 'Ass...",COURSON JAMES E,RIVERSIDE,CA,27916,2021,30690.0,2021,0,NaN,NaN,0,NaN,0,0,0,0,0,0,27916,85-000,JAMES,COURSON,NANCY,L,COURSON,US-06065-N-009715227-R-N,815240
610608,983607,1300 HIGHWAY 95 BLYTHE CA 92225,#,7,815240003,009-715-320,6065,100660192_185740491,US-06065-N-009715320-R-N,"[{'AssesseeOwnerName': 'FODNESS BRADLEY A', 'A...",FODNESS BRADLEY A,CHINO HILLS,CA,40070,2021,44052.0,2021,784,L,1995.0,0,1,0,0,2,100,0,0,40070,85-000,BRADLEY,FODNESS,TERRI,M,FODNESS,US-06065-N-009715320-R-N,815240
610609,983608,1300 HIGHWAY 95 BLYTHE CA 92225,#,8,815240003,009-715-321,6065,100660192_185740491,US-06065-N-009715321-R-N,"[{'AssesseeOwnerName': 'RUDELL RICHARD', 'Asse...",RUDELL RICHARD,VICTORVILLE,CA,35592,2021,39130.0,2021,728,L,1994.0,0,1,0,0,2,100,0,0,35592,85-000,RICHARD,RUDELL,NaN,NaN,NaN,US-06065-N-009715321-R-N,815240
610610,983609,1300 HIGHWAY 95 BLYTHE CA 92225,#,26,815240003,009-715-322,6065,100660192_185740491,US-06065-N-009715322-R-N,"[{'AssesseeOwnerName': 'TERRY JACK', 'Assessee...",TERRY JACK,FALLBROOK,CA,55354,2021,60856.0,2021,1248,L,1994.0,0,1,0,0,3,200,0,0,55354,85-000,JACK,TERRY,NaN,NaN,NaN,US-06065-N-009715322-R-N,815240


In [61]:
units[units['parcelapn'] == '001-022-002-000']

,ogr_fid,ParcelFullAddress,stunitprfx,stunitnum,parcelapn,taxapn,fips,parcelid,Upi,Assessments,AssesseeOwnerName,AssesseeMail_CityName,AssesseeMail_StateCode,TotalAssessedValue,AssessmentYear,TaxAmount,TaxYear,BuildingArea,MainBuildingAreaIndicator,YearBuilt,NumberOfBuildings,NumberOfStories,TotalNumberofRooms,NumberofUnits,NumberofBedrooms,NumberofBaths,LatestValidPrice,AssessedLandValue,AssessedImprovementValue,TaxRateCodeArea,Owner1FirstName,Owner1LastName,Owner2Firstname,Owner2MiddleName,Owner2LastName,ResoUPI
38086,5866,50 10TH ST COLUSA CA 95932,SPC,6,001-022-002-000,910-000-049-000,6011,100660192_7250525,US-06011-N-910000049000-R-N,"[{'AssesseeOwnerName': 'GUTIERREZ OTILIA', 'As...",GUTIERREZ OTILIA,WILLIAMS,CA,2000,2021,NaN,2021,680,T,1950.0,0,1,10,0,0,0,0,0,2000,1-019,OTILIA,GUTIERREZ,NaN,NaN,NaN,US-06011-N-910000049000-R-N
38087,5867,50 10TH ST COLUSA CA 95932,SPC,5,001-022-002-000,910-000-002-000,6011,100660192_7250525,US-06011-N-910000002000-R-N,"[{'AssesseeOwnerName': 'BAIRD MIKE', 'Assessee...",BAIRD MIKE,COLUSA,CA,2000,2021,NaN,2021,0,NaN,NaN,0,NaN,0,0,0,0,0,0,2000,1-019,MIKE,BAIRD,NaN,NaN,NaN,US-06011-N-910000002000-R-N


In [62]:
p_buildings[p_buildings['parcelapn'] == '001-022-002-000']

,ogr_fid,ParcelFullAddress,stunitprfx,stunitnum,parcelapn,taxapn,fips,parcelid,Upi,Assessments,AssesseeOwnerName,AssesseeMail_CityName,AssesseeMail_StateCode,TotalAssessedValue,AssessmentYear,TaxAmount,TaxYear,BuildingArea,MainBuildingAreaIndicator,YearBuilt,NumberOfBuildings,NumberOfStories,TotalNumberofRooms,NumberofUnits,NumberofBedrooms,NumberofBaths,LatestValidPrice,AssessedLandValue,AssessedImprovementValue,TaxRateCodeArea,Owner1FirstName,Owner1LastName,Owner2Firstname,Owner2MiddleName,Owner2LastName,ResoUPI
37756,5865,50 10TH ST COLUSA CA 95932,NaN,NaN,001-022-002-000,001-022-002-000,6011,100660192_7250525,US-06011-N-001022002000-R-N,[{'AssesseeOwnerName': 'BERGH OWEN H & IRENE T...,BERGH OWEN H & IRENE T,GEORGETOWN,CA,220000,2021,258030.0,2021,680,T,1950.0,0,NaN,0,0,1,100,0,155000,65000,1-019,OWEN H,BERGH,IRENE T,NaN,BERGH,US-06011-N-001022002000-R-N


In [63]:
p_buildings[p_buildings['ParcelFullAddress'].str.contains('10TH ST COLUSA')]

,ogr_fid,ParcelFullAddress,stunitprfx,stunitnum,parcelapn,taxapn,fips,parcelid,Upi,Assessments,AssesseeOwnerName,AssesseeMail_CityName,AssesseeMail_StateCode,TotalAssessedValue,AssessmentYear,TaxAmount,TaxYear,BuildingArea,MainBuildingAreaIndicator,YearBuilt,NumberOfBuildings,NumberOfStories,TotalNumberofRooms,NumberofUnits,NumberofBedrooms,NumberofBaths,LatestValidPrice,AssessedLandValue,AssessedImprovementValue,TaxRateCodeArea,Owner1FirstName,Owner1LastName,Owner2Firstname,Owner2MiddleName,Owner2LastName,ResoUPI
37756,5865,50 10TH ST COLUSA CA 95932,NaN,NaN,001-022-002-000,001-022-002-000,6011,100660192_7250525,US-06011-N-001022002000-R-N,[{'AssesseeOwnerName': 'BERGH OWEN H & IRENE T...,BERGH OWEN H & IRENE T,GEORGETOWN,CA,220000,2021,258030.0,2021,680,T,1950.0,0,NaN,0,0,1,100,0,155000,65000,1-019,OWEN H,BERGH,IRENE T,NaN,BERGH,US-06011-N-001022002000-R-N


In [64]:
units[units['ParcelFullAddress'].str.contains('PIERCE ')]

,ogr_fid,ParcelFullAddress,stunitprfx,stunitnum,parcelapn,taxapn,fips,parcelid,Upi,Assessments,AssesseeOwnerName,AssesseeMail_CityName,AssesseeMail_StateCode,TotalAssessedValue,AssessmentYear,TaxAmount,TaxYear,BuildingArea,MainBuildingAreaIndicator,YearBuilt,NumberOfBuildings,NumberOfStories,TotalNumberofRooms,NumberofUnits,NumberofBedrooms,NumberofBaths,LatestValidPrice,AssessedLandValue,AssessedImprovementValue,TaxRateCodeArea,Owner1FirstName,Owner1LastName,Owner2Firstname,Owner2MiddleName,Owner2LastName,ResoUPI
0,1,535 PIERCE ST ALBANY CA 94706,APT,3115,66-2763-16,66-2763-16,6001,509725994_16,US-06001-N-66276316-R-N,[{'AssesseeOwnerName': 'CHINGCUANCO ALFREDO & ...,CHINGCUANCO ALFREDO & LAUDENCIACHINGCUANCO DEBBIE,ALBANY,CA,594024,2021,1053740.0,2021,1170,L,1988.0,1,4.0,5,1,2,200,670000,178207,415817,22-000,ALFREDO,CHINGCUANCO,DEBBIE,NaN,LAUDENCIA CHINGCUANCO,US-06001-N-66276316-R-N
1,2,535 PIERCE ST ALBANY CA 94706,APT,3116,66-2763-17,66-2763-17,6001,509725994_17,US-06001-N-66276317-R-N,[{'AssesseeOwnerName': 'MEI BAISONG & JIN FANG...,MEI BAISONG & JIN FANG,WABAN,MA,298029,2021,655476.0,2021,1170,L,1988.0,1,4.0,5,235,2,200,210000,89451,208578,22-000,BAISONG,MEI,FANG,NaN,JIN,US-06001-N-66276317-R-N
2,3,535 PIERCE ST ALBANY CA 94706,APT,3114,66-2763-15,66-2763-15,6001,509725994_15,US-06001-N-66276315-R-N,[{'AssesseeOwnerName': 'BHATTARAI NARAYAN & PU...,BHATTARAI NARAYAN & PUSHPA L,ALBANY,CA,595000,2021,1055086.0,2021,1170,L,1988.0,1,4.0,5,235,2,200,595000,178500,416500,22-000,NARAYAN,BHATTARAI,PUSHPA L NEPAL,NaN,BHATTARAI,US-06001-N-66276315-R-N
3,4,535 PIERCE ST ALBANY CA 94706,APT,3201,66-2763-19,66-2763-19,6001,509725994_19,US-06001-N-66276319-R-N,[{'AssesseeOwnerName': 'CHAN DICKON T & MA LIW...,CHAN DICKON T & MA LIWEN,ALBANY,CA,630701,2021,1104286.0,2021,1170,L,1988.0,1,4.0,5,235,2,200,600000,189210,441491,22-000,DICKON TAK YAN,CHAN,LIWEN,NaN,MA,US-06001-N-66276319-R-N
4,5,535 PIERCE ST ALBANY CA 94706,APT,3113,66-2763-14,66-2763-14,6001,509725994_14,US-06001-N-66276314-R-N,"[{'AssesseeOwnerName': 'LIU TERESA', 'Assessee...",LIU TERESA,ALBANY,CA,310975,2021,673318.0,2021,1170,L,1988.0,1,4.0,5,235,2,200,268000,93292,217683,22-000,TERESA,LIU,NaN,NaN,NaN,US-06001-N-66276314-R-N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
889652,69602,509 PIERCE ST SAN FRANCISCO CA 94117,APT,4,0824022,0824 -022,6075,511317412_5215,US-06075-N-0824022-R-N,[{'AssesseeOwnerName': 'HANONO GERALD BENJAMIN...,HANONO GERALD BENJAMIN,SAN FRANCISCO,CA,951000,2021,1204064.0,2021,811,L,1924.0,0,1,0,1,0,100,1111000,475500,475500,1-000,NaN,509 PIERCE 4 LLC,NaN,NaN,NaN,US-06075-N-0824022-R-N
889680,69793,356 PIERCE ST SAN FRANCISCO CA 94117,APT,1,0844041,0844 -041,6075,511317412_5395,US-06075-N-0844041-R-N,"[{'AssesseeOwnerName': 'TAROWSKY NATALIE', 'As...",TAROWSKY NATALIE,SAN FRANCISCO,CA,138100,2021,234544.0,2021,517,L,1907.0,0,2,0,1,0,0,90000,46029,92071,1-000,NATALIE,TAROWSKY,NaN,NaN,NaN,US-06075-N-0844041-R-N
889681,69794,358 PIERCE ST SAN FRANCISCO CA 94117,#,2,0844042,0844 -042,6075,511317412_5396,US-06075-N-0844042-R-N,[{'AssesseeOwnerName': 'JOHN PETER BENEDETTO T...,JOHN PETER BENEDETTO TRUST,SAN FRANCISCO,CA,285440,2021,408770.0,2021,1087,L,1907.0,0,2,0,1,0,0,186000,122768,162672,1-000,JOHN PETER,BENEDETTO,NaN,NaN,THE JOHN PETER BENEDETTO TRUST,US-06075-N-0844042-R-N
889682,69795,356 PIERCE ST SAN FRANCISCO CA 94117,APT,3,0844043,0844 -043,6075,511317412_5397,US-06075-N-0844043-R-N,[{'AssesseeOwnerName': 'DEBAIGUE ARTHUR KENZO'...,DEBAIGUE ARTHUR KENZO,SAN FRANCISCO,CA,813338,2021,1041282.0,2021,718,L,1907.0,0,1,3,1,1,100,805000,406669,406669,1-000,ARTHUR KENZO,DEBAIGUE,NaN,NaN,NaN,US-06075-N-0844043-R-N


In [65]:
p_buildings[p_buildings['ParcelFullAddress'].str.contains('PIERCE ')]

,ogr_fid,ParcelFullAddress,stunitprfx,stunitnum,parcelapn,taxapn,fips,parcelid,Upi,Assessments,AssesseeOwnerName,AssesseeMail_CityName,AssesseeMail_StateCode,TotalAssessedValue,AssessmentYear,TaxAmount,TaxYear,BuildingArea,MainBuildingAreaIndicator,YearBuilt,NumberOfBuildings,NumberOfStories,TotalNumberofRooms,NumberofUnits,NumberofBedrooms,NumberofBaths,LatestValidPrice,AssessedLandValue,AssessedImprovementValue,TaxRateCodeArea,Owner1FirstName,Owner1LastName,Owner2Firstname,Owner2MiddleName,Owner2LastName,ResoUPI
55,100565,5000 PIERCE RD BAKERSFIELD CA 93308,NaN,NaN,11607012,116-160-34-00-0,6029,100660192_173633878,US-06029-N-11616034000-R-N,"[{'AssesseeOwnerName': 'NICHELSON MICHAEL A', ...",NICHELSON MICHAEL A,BAKERSFIELD,CA,5658,2021,NaN,2021,720,L,1973.0,0,1,0,0,2,100,0,0,5658,57-827,MICHAEL,NICHELSON,NaN,NaN,NaN,US-06029-N-11616034000-R-N
56,100578,5000 PIERCE RD BAKERSFIELD CA 93308,NaN,NaN,11607012,116-070-12-00-0,6029,100660192_173633878,US-06029-N-11607012000-R-N,"[{'AssesseeOwnerName': 'BKMMMHP LLC', 'Assesse...",BKMMMHP LLC,ARROYO GRANDE,CA,998958,2021,2563626.0,2021,811,T,1947.0,0,NaN,0,40,0,0,844000,880602,118356,57-027,NaN,BKMMMHP LLC,NaN,NaN,NaN,US-06029-N-11607012000-R-N
963,971395,69353 PIERCE ST THERMAL CA 92274,NaN,NaN,749140016,749-140-016,6065,100660192_185729067,US-06065-N-749140016-R-N,"[{'AssesseeOwnerName': 'GAMEZ PARTNERS', 'Asse...",GAMEZ PARTNERS,THERMAL,CA,322129,2021,424270.0,2021,1152,T,2016.0,2,1,0,2,0,0,0,40336,281793,58-038,NaN,GAMEZ PARTNERS,NaN,NaN,NaN,US-06065-N-749140016-R-N
2055,973118,76250 PIERCE ST THERMAL CA 92274,NaN,NaN,755180001,755-180-001,6065,100660192_185730731,US-06065-N-755180001-R-N,"[{'AssesseeOwnerName': 'MORA MARTHA', 'Assesse...",MORA MARTHA,THERMAL,CA,387911,2021,661882.0,2021,0,NaN,NaN,0,NaN,0,0,0,0,0,310343,77568,58-038,MARTHA,MORA,NaN,NaN,NaN,US-06065-N-755180001-R-N
11414,94915,4000 PIERCE ST RIVERSIDE CA 92505,NaN,NaN,142080028,142-080-028,6065,100660192_184986442,US-06065-N-142080028-R-N,[{'AssesseeOwnerName': 'RIVERSIDE MEADOWS LTD'...,RIVERSIDE MEADOWS LTD,NEWPORT BEACH,CA,3567659,2021,6177804.0,2021,1440,T,1972.0,0,1,0,0,0,0,0,1016076,2551583,9-176,NaN,RIVERSIDE MEADOWS LTD,NaN,NaN,NaN,US-06065-N-142080028-R-N
11415,95798,4000 PIERCE ST RIVERSIDE CA 92505,NaN,NaN,142170001,142-170-001,6065,100660192_184987229,US-06065-N-142170001-R-N,[{'AssesseeOwnerName': 'RIVERSIDE MEADOWS LTD'...,RIVERSIDE MEADOWS LTD,NEWPORT BEACH,CA,1054847,2021,1419580.0,2021,0,NaN,NaN,0,NaN,0,0,0,0,0,339103,715744,9-176,NaN,RIVERSIDE MEADOWS LTD,NaN,NaN,NaN,US-06065-N-142170001-R-N
13451,319,545 PIERCE ST ALBANY CA 94706,NaN,NaN,66-2761-126,66-2761-126,6001,100660192_173145133,US-06001-N-662761126-R-N,[{'AssesseeOwnerName': 'NOT AVAILABLE FROM THE...,NOT AVAILABLE FROM THE COUNTY,ALBANY,CA,0,2021,NaN,2021,153918,L,1984.0,1,4,0,103,0,2000,0,0,0,22-000,NaN,NOT AVAILABLE FROM THE COUNTY,NaN,NaN,NaN,US-06001-N-662761126-R-N
21436,320,555 PIERCE ST ALBANY CA 94706,NaN,NaN,66-2761-13,66-2761-13,6001,100660192_173145134,US-06001-N-66276113-R-N,[{'AssesseeOwnerName': 'NOT AVAILABLE FROM THE...,NOT AVAILABLE FROM THE COUNTY,ALBANY,CA,0,2021,NaN,2021,0,NaN,NaN,0,NaN,0,0,0,0,0,0,0,22-000,NaN,NOT AVAILABLE FROM THE COUNTY,NaN,NaN,NaN,US-06001-N-66276113-R-N
24403,40,535 PIERCE ST ALBANY CA 94706,NaN,NaN,66-2761-129,66-2761-129,6001,100660192_173145050,US-06001-N-662761129-R-N,[{'AssesseeOwnerName': 'COMMON ARE OF TR5873 (...,COMMON ARE OF TR5873 (66-2763) 1THRU235 & 131T...,BURLINGAME,CA,0,2021,NaN,2021,317904,L,1988.0,1,4,0,235,0,0,221000,0,0,22-000,NaN,NaN,NaN,NaN,NaN,US-06001-N-662761129-R-N
24404,43,535 PIERCE ST ALBANY CA 94706,NaN,NaN,66-2761-128,66-2761-128,6001,100660192_173145049,US-06001-N-662761128-R-N,[{'AssesseeOwnerName': 'BAYSIDE COMMONS CONDOM...,BAYSIDE COMMONS CONDOMINIUM OWNERS ASSOCIATION,BURLINGAME,CA,0,2021,NaN,2021,0,NaN,NaN,0,NaN,0,0,0,0,147500,0,0,22-000,NaN,BAYSIDE COMMONS CONDOMINIUM OWNERS ASSOCIATION,NaN,NaN,NaN,US-06001-N-662761128-R-N


In [66]:
units['key'] = [str(x).replace('-','')[:6] for x in units['parcelapn']]

In [67]:
units.groupby('ParcelFullAddress')['key'].nunique()[units.groupby('ParcelFullAddress')['key'].nunique() > 1]

ParcelFullAddress
1  APPIAN WAY SOUTH SAN FRANCISCO CA 94080        5
1  BAKER ST SAN FRANCISCO CA 94117                2
1  BALDWIN AVE SAN MATEO CA 94401                 5
1  BLUXOME ST SAN FRANCISCO CA 94107              7
1  CIVIC CENTER DR SAN MARCOS CA 92069            2
                                                 ..
I-280  N  SAN FRANCISCO CA 94124                  3
I-280  S  SAN FRANCISCO CA 94107                  2
I-280  S  SAN FRANCISCO CA 94112                  2
SHERMAN ISLAND EAST LEVE RD RIO VISTA CA 94571    2
SOLSTICE AVE CHULA VISTA CA 91915                 3
Name: key, Length: 4711, dtype: int64

In [68]:
units.groupby('ParcelFullAddress')['parcelapn'].nunique()[units.groupby('ParcelFullAddress')['parcelapn'].nunique() > 1]

ParcelFullAddress
1  3RD PL LONG BEACH CA 90802                    42
1  62ND PL LONG BEACH CA 90803                   12
1  ANDREW DR BELVEDERE TIBURON CA 94920           5
1  ANDREW DR TIBURON CA 94920                     3
1  APPIAN WAY SOUTH SAN FRANCISCO CA 94080      145
                                               ... 
SOLSTICE AVE CHULA VISTA CA 91915                12
SUGAR BUSH CT CHULA VISTA CA 91915                2
THE EMBARCADERO PIER  SAN FRANCISCO CA 94111      2
VIA CANDIDIZ  SAN DIEGO CA 92130                  2
W HARBOR DR SAN DIEGO CA 92101                    2
Name: parcelapn, Length: 70445, dtype: int64

In [69]:
units.groupby('ParcelFullAddress')['parcelapn'].nunique()

ParcelFullAddress
094 # 94 OFC HWY CAMPO CA 91906           1
1    KETTLEMAN CITY CA 93239              1
1    LAKESIDE CA 92040                    1
1  1ST ST LAKEPORT CA 95453               1
1  3RD PL LONG BEACH CA 90802            42
                                         ..
W11-B  HARBOR DR SAN DIEGO CA 92101       1
W14-F1  WESTERN ST SAN DIEGO CA 92101     1
W4-C  WESTERN ST SAN DIEGO CA 92027       1
W7-A  WESTERN ST SAN DIEGO CA 92101       1
WEST OF HWY   93526                       1
Name: parcelapn, Length: 170659, dtype: int64

In [70]:
units[units['ParcelFullAddress'] == '1  APPIAN WAY SOUTH SAN FRANCISCO CA 94080']

,ogr_fid,ParcelFullAddress,stunitprfx,stunitnum,parcelapn,taxapn,fips,parcelid,Upi,Assessments,AssesseeOwnerName,AssesseeMail_CityName,AssesseeMail_StateCode,TotalAssessedValue,AssessmentYear,TaxAmount,TaxYear,BuildingArea,MainBuildingAreaIndicator,YearBuilt,NumberOfBuildings,NumberOfStories,TotalNumberofRooms,NumberofUnits,NumberofBedrooms,NumberofBaths,LatestValidPrice,AssessedLandValue,AssessedImprovementValue,TaxRateCodeArea,Owner1FirstName,Owner1LastName,Owner2Firstname,Owner2MiddleName,Owner2LastName,ResoUPI,key
939310,51295,1 APPIAN WAY SOUTH SAN FRANCISCO CA 94080,UNIT,713-10,100520400,100-520-400,6081,100660192_163202437,US-06081-N-100520400-R-N,"[{'AssesseeOwnerName': 'ABAYA RANDOLPH R', 'As...",ABAYA RANDOLPH R,SOUTH SAN FRANCISCO,CA,280417,2021,293828.0,2021,1043,L,1979.0,0,2,4,150,2,200,0,56071,224346,13-015,RANDOLPH R,ABAYA,NaN,NaN,NaN,US-06081-N-100520400-R-N,100520
939311,51297,1 APPIAN WAY SOUTH SAN FRANCISCO CA 94080,UNIT,711-3,100520350,100-520-350,6081,100660192_163202432,US-06081-N-100520350-R-N,[{'AssesseeOwnerName': 'DUQUE ARTHUR-SAM A & J...,DUQUE ARTHUR-SAM A & JOYCE R,DALY CITY,CA,177533,2021,192394.0,2021,777,L,1979.0,0,1,3,150,1,100,150000,53259,124274,13-015,ARTHUR SAM A,DUQUE,JOYCE R,NaN,DUQUE,US-06081-N-100520350-R-N,100520
939312,51299,1 APPIAN WAY SOUTH SAN FRANCISCO CA 94080,UNIT,712-2,100510150,100-510-150,6081,100660192_163202386,US-06081-N-100510150-R-N,[{'AssesseeOwnerName': 'LAI PHILIP M & VALERIE...,LAI PHILIP M & VALERIE C,SOUTH SAN FRANCISCO,CA,171257,2021,178348.0,2021,1357,L,1979.0,0,1,5,150,3,200,0,34239,137018,13-015,PHILIP M,LAI,VALERIE C,NaN,LAI,US-06081-N-100510150-R-N,100510
939313,51300,1 APPIAN WAY SOUTH SAN FRANCISCO CA 94080,UNIT,712-1,100510160,100-510-160,6081,100660192_163202387,US-06081-N-100510160-R-N,"[{'AssesseeOwnerName': 'LEUNG WALTER YUKSZE', ...",LEUNG WALTER YUKSZE,SOUTH SAN FRANCISCO,CA,612436,2021,645072.0,2021,1357,L,1979.0,0,1,5,150,3,200,560000,183730,428706,13-015,WALTER YUKSZE,LEUNG,LING SIU,NaN,NGAI,US-06081-N-100510160-R-N,100510
939314,51302,1 APPIAN WAY SOUTH SAN FRANCISCO CA 94080,UNIT,713-2,100510170,100-510-170,6081,100660192_163202388,US-06081-N-100510170-R-N,"[{'AssesseeOwnerName': 'HOUSE STEVEN K', 'Asse...",HOUSE STEVEN K,SOUTH SAN FRANCISCO,CA,662241,2021,705166.0,2021,1357,L,1979.0,0,1,5,150,3,200,630000,199723,462518,13-015,STEVEN K,HOUSE,KAREN R,NaN,CURELLA,US-06081-N-100510170-R-N,100510
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939450,51507,1 APPIAN WAY SOUTH SAN FRANCISCO CA 94080,UNIT,702-5,100510020,100-510-020,6081,100660192_163202373,US-06081-N-100510020-R-N,"[{'AssesseeOwnerName': 'CHIN WINFRED', 'Assess...",CHIN WINFRED,SOUTH SAN FRANCISCO,CA,156648,2021,170298.0,2021,720,L,1979.0,0,1,3,150,1,100,135000,46993,109655,13-015,WINFRED,CHIN,NaN,NaN,NaN,US-06081-N-100510020-R-N,100510
939451,51508,1 APPIAN WAY SOUTH SAN FRANCISCO CA 94080,UNIT,702-8,100520040,100-520-040,6081,100660192_163202401,US-06081-N-100520040-R-N,"[{'AssesseeOwnerName': 'KWONG SANDY CHE', 'Ass...",KWONG SANDY CHE,SOUTH SAN FRANCISCO,CA,660000,2021,695390.0,2021,1030,L,1979.0,0,2,4,150,2,150,660000,198000,462000,13-015,SANDY CHE,KWONG,NaN,NaN,NaN,US-06081-N-100520040-R-N,100520
939452,51513,1 APPIAN WAY SOUTH SAN FRANCISCO CA 94080,UNIT,702-11,100520020,100-520-020,6081,100660192_163202399,US-06081-N-100520020-R-N,"[{'AssesseeOwnerName': 'LEW JACK ZIM TR', 'Ass...",LEW JACK ZIM TR,SAN FRANCISCO,CA,157603,2021,171310.0,2021,777,L,1979.0,0,1,3,150,1,100,0,31502,126101,13-015,JACK ZIM,LEW,NaN,NaN,JACK ZIM LEW FAMILY TRUST,US-06081-N-100520020-R-N,100520
939453,51515,1 APPIAN WAY SOUTH SAN FRANCISCO CA 94080,UNIT,702-14,100540020,100-540-020,6081,100660192_163202478,US-06081-N-100540020-R-N,"[{'AssesseeOwnerName': 'GAVIDIA CHRISTOPHER', ...",GAVIDIA CHRISTOPHER,SOUTH SAN FRANCISCO,CA,591060,2021,622458.0,2021,1032,L,1979.0,0,2,4,150,2,100,558000,186916,404144,1

There are two cases for units:
1. parcelapn is correct
2. taxapn == parcelapn

Should we include communities as buildings complexes? When grouping by parcelapn in potential_buildings there are instances where some of the buildings in the neighborhood share a parcel apn but the rest don't (schubert ct - University Hills, Irvine)